# Watson Assistant chatbot service on IBM Cloud: Final Exam Project
## Group 13

## Import Libraries

In [1]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import AssistantV2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import math
import cplex
import ipopt

C:\Users\arshb\anaconda3\lib\site-packages\ipopt\__init__.py:13: FutureWarning: The module has been renamed to 'cyipopt' from 'ipopt'. Please import using 'import cyipopt' and remove all uses of 'import ipopt' in your code as this will be deprecated in a future release.
  warnings.warn(msg, FutureWarning)


## Helper Functions

In [2]:
def rebalance(todayprice, old_x, W_new, Cash_old, tr):
    Cash=False
    
    #compute current portfolio value
    V_cur = np.dot(todayprice, old_x) + Cash_old
    W_new = np.asarray(W_new)
    #calculate number of stocks to buy and sell
    new_x = np.divide(np.multiply(V_cur,W_new), todayprice)
    new_x = rounder_vec(new_x) #round up or down up when first decimal is 0.5 or above and round down otherwise
    
 
    #rebalancing loop
    while Cash==False:
        #calculate buy or sell
        change_x = np.subtract(new_x, old_x) #if positive means buy if negative means sell
        tran_buyl = []
        tran_selll = []
        for i in change_x:
            if i>=0: #making a list of all the stocks which are to buy
                tran_buyl.append(1)
                tran_selll.append(0)
            if i<0: #making a list of all the stocks which are to sell
                tran_buyl.append(0)
                tran_selll.append(1)
        tran_buyl =  np.asarray(tran_buyl) #list with value 1 are to buy stocks
        tran_selll =  np.asarray(tran_selll) #list with value 1 are to sell stocks
        
        tran_buy = np.sum(new_x * tran_buyl * todayprice) * tr #total transaction cost for buy the stocks
        tran_sell = np.sum(new_x * tran_selll * todayprice) * tr #total transaction cost for sell the stocks
        
        Newvalue = np.dot(new_x, todayprice) + tran_buy - tran_sell #total value of the portfolio adding buying cost and subtracting sell cost
              
        Cash_new = V_cur - Newvalue #balancing equation (current portfolio value - new portfolio value + cash account)
        
        #balacing the cash account so its never below zero
        
        if Cash_new >= 0: #if cash account positive then break the rebalancing loop and return the cash and x optimal
            Cash == True
            break
         
        if Cash_new < 0: #if cash negative then recalculate x optimal
            newlist = (np.multiply(new_x, todayprice) - abs(Cash_new)).tolist() #which stock has a value more than the cash account
            idx_list = np.array(newlist).argsort().tolist() #listing the stocks index in a ascending order
            
            for i in idx_list: #checking the stock value by lowest to highest
                if newlist[i] > 0: #if the stock value is above zero and very close to cash account balance then remove that stock
                    new_x[i] = new_x[i] - 1 #removing the stock which has a close value to the abs(negative cash value). Minimizing the impact on the overall portfolio
                    break
                                                
    return new_x, Cash_new

def rounder(x): #parts of the code are from https://stackoverflow.com/questions/53201470/how-to-always-round-up-a-xx-5-in-numpy
    if (x-int(x) >= 0.5): #round the asset up if the secoond decimal place is equal or greater that 0.5
        return np.ceil(x)
    else: 
        return np.floor(x) #round the asset down otherwise
rounder_vec = np.vectorize(rounder) #vectorize to apply the function to every value in array

### Equally Weighted

In [3]:
def strat_equally_weighted(x_init, cash_init, mu, Q, cur_prices, tr):
    interest = 0
    #compute current portfolio value
    V_cur = np.dot(cur_prices, x_init) + cash_init 
    w = 1/len(x_init) #weights to be equal amoung all assets for equally weighted strategy
    x_optimal, cash_optimal = rebalance(cur_prices, x_init, w, cash_init, tr) #applying rebalancing algorithim to balance and 
                                                                          #optimize the number of stocks and cash account balance.
    return x_optimal, cash_optimal,interest

### Minimum Variance

In [4]:
def strat_min_variance(x_init, cash_init, mu, Q, cur_prices, tr):
    np.random.seed(978351)
    
    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize) #setting cplex to minimize
    n = len(x_init)
    c  = [0.0] * n #no linear part in the equation
    lb = [0.0] * n # w>= 0
    ub = [1.0] * n #upper bound = 1
    
    A = [] #LHS of the constraints 
    for k in range(n):
        A.append([[0, 1],[1.0, 0]])
    
    var_names = ["w_%s" % i for i in range(1,n+1)] #total number of variables
    
    cpx.linear_constraints.add(rhs=[1.0, 0], senses="EG") #RHS of the constraints
    cpx.variables.add(obj=c, lb=lb, ub=ub, columns=A, names=var_names) #adding the linear part of the equation into the solver
    
    Qmat = [[list(range(n)), list(2*Q[k,:])] for k in range(n)] #defining the quadratic part of the equations
    
    
    cpx.objective.set_quadratic(Qmat) #adding the quadratic part of the solver
    
    cpx.parameters.threads.set(4)
    
    # Disable CPLEX output to screen
    cpx.set_results_stream(None)
    cpx.set_warning_stream(None)
    
    # Compute min varuabce solution with CPLEX
    cpx.solve()
    w_cur = cpx.solution.get_values()
      
    x_optimal, cash_optimal = rebalance(cur_prices, x_init, w_cur, cash_init, tr)#applying rebalancing algorithim to balance and 
                                                                          #optimize the number of stocks and cash account balance.
    
    intrest = 0
    return x_optimal, cash_optimal, intrest

### Maximum Sharpe Ratio

In [5]:
# MAX SHARPE RATIO
def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices, tr):
    
    interest = 0
    portfolio_value = cur_prices.dot(x_init) + cash_init

    
    n1 = len(x_init)
    n = n1+1
    r_rf = 0.025
    
    daily_rf = r_rf / 252

    Q1 = np.append(Q,np.zeros((n1,1)),axis=1)
    new_row = np.zeros((n))
    Q2 = np.vstack([Q1,new_row]) #add new column and new row for risk-free asset

    diff = mu-daily_rf * np.ones(n1) # for convenience

    A = []
    for k in range(n-1):
        A.append([[0,1],[diff[k],1.0]])
    A.append([[0,1],[0,-1.0]]) # diff.dot(y)==1

    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)
    c = [0]*n
    lb = [0]*n
    ub = [np.inf]*n

    var_names = ['y_%s'% i for i in range(1,n+1)]
    cpx.linear_constraints.add(rhs=[1.0,0],senses='EE')
    cpx.variables.add(obj=c,lb=lb,ub=ub,columns=A,names=var_names)

    qmat = [[list(range(n)),list(2*Q2[k,:])] for k in range(n)]

    cpx.objective.set_quadratic(qmat)
    cpx.parameters.threads.set(6)
    cpx.set_results_stream(None)
    cpx.set_warning_stream(None)
    cpx.solve()
    
    if cpx.solution.get_status_string()== 'infeasible':
            x_optimal = x_init
            cash_optimal = cash_init
            w_maxSharp = (x_init*cur_prices)/portfolio_value
            
    else:
        result = np.array(cpx.solution.get_values())
        w_maxSharp = result[0:n1]/result[n1]

        x_optimal, cash_optimal = rebalance(cur_prices, x_init, w_maxSharp, cash_init, tr)#applying rebalancing algorithim to balance and 
                                                                              #optimize the number of stocks and cash account balance.
    
    return x_optimal, cash_optimal,interest

### Equal Risk Contribution

In [6]:
class erc(object):
    def __init__(self):
        pass

    def objective(self, x):
        # The callback for calculating the objective
        y = x * np.dot(Q, x)
        fval = 0
        n = len(x)
        for i in range(n):
            for j in range(i,n):
                xij = y[i] - y[j]
                fval = fval + xij*xij
        fval = 2*fval
        return fval

    def gradient(self, x):
        # The callback for calculating the gradient
        a = np.dot(Q,x)
        b = np.multiply(a, x)
        c = sum(-1*b)
        grad = np.zeros(len(x))
        
        for  i in range(0, len(x)):
            grad[i] = 2*a[i]*(c+len(x)*b[i])
        return grad

    def constraints(self, x):
        # The callback for calculating the constraints
        return [1.0] * len(x)
    
    def jacobian(self, x):
        # The callback for calculating the Jacobian
        return np.array([[1.0] * len(x)])

In [7]:
def strat_equal_risk_contr(x_init, cash_init, mu, Q, cur_prices, tr):
    n = len(x_init) 
    w0 = [1.0/n] * n
    # Use "1/n portfolio" w0 as initial portfolio for starting IPOPT optimization

    lb = [0.0] * n  # lower bounds on variables
    ub = [1.0] * n  # upper bounds on variables
    cl = [1]        # lower bounds on constraints
    cu = [1]        # upper bounds on constraints

    # Define IPOPT problem
    nlp = ipopt.Problem(n=len(x_init), m=len(cl), problem_obj=erc(), lb=lb, ub=ub, cl=cl, cu=cu)
 
    # Set the IPOPT options
    nlp.add_option('jac_c_constant'.encode('utf-8'), 'yes'.encode('utf-8'))
    nlp.add_option('hessian_approximation'.encode('utf-8'), 'limited-memory'.encode('utf-8'))
    nlp.add_option('mu_strategy'.encode('utf-8'), 'adaptive'.encode('utf-8'))
    nlp.add_option('tol'.encode('utf-8'), 1e-10)

    # Solve the problem
    w_erc, info = nlp.solve(w0)
    
    # Compute variance and asset risk contributions for the ERC portfolio
    var_ERC = np.dot(w_erc, np.dot(Q, w_erc))
    std_ERC = np.sqrt(var_ERC)
    RC_ERC = (w_erc * np.dot(Q, w_erc)) / std_ERC
    
  
    if np.allclose(np.sum(w_erc),1) and np.allclose(np.sum(RC_ERC), std_ERC): #check to see if equal variaence and sum of weights
        x_optimal, cash_optimal = rebalance(cur_prices, x_init, w_erc, cash_init, tr)#applying rebalancing algorithim to balance and 
                                                                          #optimize the number of stocks and cash account balance.
    
    interest = 0
    return x_optimal, cash_optimal, interest

### Robust Mean-Variance

In [8]:
def strat_robust_optim(x_init, cash_init, mu, Q, cur_prices, tr):
    
    n = len(x_init)
    w0 = [1/n]*n
    var_matr = np.diag(np.diag(Q))
    rob_init = np.dot(w0, np.dot(var_matr, w0))
    rob_bnd = rob_init #target variance to equal to equally weighted 
    
    r_rf = 0.025
    
    daily_rf = r_rf/252
    
    Portf_Retn = daily_rf #target portfolio return to equal daily  risk free rate of return
        
    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    c  = [0.0] * n
    lb = [0.0] * n
    ub = [1.0] * n    
    
    A = []
    for k in range(n):
        A.append([[0,1],[1.0,mu[k]]])
    
    var_names = ["w_%s" % i for i in range(1,n+1)]
    
    cpx.linear_constraints.add(rhs=[1.0,Portf_Retn], senses="EG")
    
    cpx.variables.add(obj=c, lb=lb, ub=ub, columns=A, names=var_names)
    
    Qmat = [[list(range(n)), list(2*Q[k,:])] for k in range(n)]
    
    cpx.objective.set_quadratic(Qmat)
    
    Qcon = cplex.SparseTriple(ind1=var_names, ind2=range(n), val=np.diag(var_matr))
    cpx.quadratic_constraints.add(rhs=rob_bnd, quad_expr=Qcon, name="Qc")
    
    cpx.parameters.threads.set(4)
    cpx.parameters.timelimit.set(60)
    cpx.parameters.barrier.qcpconvergetol.set(1e-12)
    
    # Disable CPLEX output to screen
    cpx.set_results_stream(None)
    cpx.set_warning_stream(None)
    cpx.solve()
    
    if cpx.solution.get_status_string()== 'infeasible': #if no cplex solution, keep the portfolio same as the last period
            
            x_optimal = x_init
            cash_optimal = cash_init
    else:
        
        w_rMV = np.array(cpx.solution.get_values()) 
        w_rMV[w_rMV<1e-7] = 0 #zeroing the weights which are closer to zero
        w_rMV = w_rMV / np.sum(w_rMV) #reassigining the weights
       
        x_optimal, cash_optimal = rebalance(cur_prices, x_init, w_rMV, cash_init, tr)#applying rebalancing algorithim to balance and 
                                                                          #optimize the number of stocks and cash account balance.
    borrow = 0
    
    return x_optimal, cash_optimal, borrow

## Clean Watson Assistant Responses

In [9]:
def clean_dic(dic):
    stocks = []
    industry = []
    Tech = ['Technology', 'Tech', 'tech']
    Auto = ['Auto', 'Car', 'Automobile', 'auto']
    Pharma = ['Pharma', 'Pharmaceutical', 'Pharama', 'pharma']
    Energy = ['Energy', 'Oil', 'Gas', 'energy']
    Banking = ['Bank', 'Banks', 'Banking', 'banking', 'bank']
    rebalance_option = ['Every month','Every 2 months', 'Quarterly', 'Half-yearly']
    strategy = 'Mininum Variance Portfolio'
    rb_options = [1,2,4,6]
    time_options = ['6 months','1 year','2 year','3 year']
    time_op = [0.5,1,2,3]
    
    if 'stocks' in dic:
        for i in range(0, len(dic['stocks'][0].split(","))):
            stocks.append(int(dic['stocks'][0].split(",")[i])) 
        invest = None
    
    if 'investamount' in dic:
        invest = int(dic['investamount'][0])
        stock = None
        
    if len(dic['industry']) == 1:
        lnd = dic['industry'][0].split(",")
    else:
        lnd = list(dic['industry'][1].split(","))
        lnd.append(dic['industry'][0])

    for i in range(0, len(lnd)):
        #Tech
        for p in Tech:
            if lnd[i] == p:
                industry.append('Tech')
        #Auto
        for p in Auto:
            if lnd[i] == p:
                industry.append('Auto')
                
        #Pharma
        for p in Pharma:
            if lnd[i] == p:
                industry.append('Pharma')

        #Energy
        for p in Energy:
            if lnd[i] == p:
                industry.append('Energy')
        
        #Bank
        for p in Banking:
            if lnd[i] == p:
                industry.append('Banking')        
    
    industry[:] = list(set(industry))
    
    #rebalance_time
    for rb in range(0, len(rebalance_option)):
        if dic['rebalance_period'][0] == rebalance_option[rb]:
            rebalance_time = rb_options[rb]
    #time_invest
    for tb in range(0, len(time_options)):
        if dic['time_period'][0] == time_options[tb]:
            time = time_op[tb]
    
    #risk
    risk = dic['risk'][0]
    

    #strategy selection
    #longtime High risk high rebalance: Sharpe Ratio
    if rebalance_time == 1 or rebalance_time == 2:
        if time == 2 or time == 3:
            if risk == "high":
                strategy = 'Maximum Sharpe Ratio Portfolio'
    
    #longtime Med risk low rebalance: Equally Weighted
    if rebalance_time == 4 or rebalance_time == 6:
        if time == 2 or time == 3:
            if risk == "moderate":
                strategy = 'Equally Weighted Portfolio'
    
    #longtime Low risk low rebalance: Equal Risk Contribution
    if rebalance_time == 4 or rebalance_time == 6:
        if time == 2 or time == 3:
            if risk == "low":
                strategy = 'Equal Risk Contributions Portfolio'
    
    #shorttime High risk high rebalance: Sharpe Ratio
    if rebalance_time == 1 or rebalance_time == 2:
        if time == 0.5 or time == 1:
            if risk == "high":
                strategy = 'Maximum Sharpe Ratio Portfolio'
    
    #shorttime Med risk low rebalance: Robust Min Variance
    if rebalance_time == 4 or rebalance_time == 6:
        if time == 0.5 or time == 1:
            if risk == "moderate":
                strategy = 'Robust Optimization Portfolio'
    
    #shorttime Low risk low rebalance: Minimum Variance
    if rebalance_time == 4 or rebalance_time == 6:
        if time == 0.5 or time == 1:
            if risk == "low":
                strategy = 'Mininum Variance Portfolio'
      
    #trans
    trans = int(dic['transactioncost'][0].split("%")[0])/100
    
    print('Existing Stocks:', stocks)
    print('Investment Amount: ',invest)
    print('Industry to invest in: ', industry)
    print('Selected Strategy: ', strategy)
    print('Rebalancing Time: ', rebalance_time)
    print('Time to invest for: ', time)
    print('Risk Level: ', risk)
    print('Transaction Rate: ', trans)
        
    return stocks, invest, industry, strategy, trans

## Portfolio Optimization

In [10]:
data = pd.read_csv('MIE1622_FinalData.csv')
data = data.dropna()

In [11]:
# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    temp = [int(x) for x in datestr.split('-')]
    return [temp[-1], temp[0], temp[1]]

In [12]:
def protopt(stocks, invest, industry, strategy, trans, data):
    ind_dic = {"Banking": ['RY', 'CM-PQ.TO', 'C', 'TD', 'JPM'], "Tech": ['AAPL', 'AMZN', 'GOOG','ORCL', 'TSLA'], 
              "Energy": ['CNQ', 'SU', 'IMO', 'ENB', 'TRP'], "Auto": ['GM', 'HMC', 'F','TM', 'TTM'],
              "Pharma": ['PFE', 'ABBV', 'JNJ', 'SNY', 'MRK']}
    name_dic = {'RY': "RoyalBank",'CM-PQ.TO': "CIBC",'C': "CITI",'TD': "TDCanada",'JPM': "JPMChase",'AAPL': "Apple",
        'AMZN': "Amazon",'GOOG': "Google",'ORCL': "Oracle",'TSLA': "Tesla",'CNQ': "CanadianNaturalResoruces",
        'SU': "SuncorEnergy",'IMO': "ImperialOil",'ENB': "Enbridge",'TRP': "TCEnergy",'GM': "GeneralMotors",
        'HMC': "Honda",'F': "Ford",'TM': "Toyota",'TTM': "TataMotors",'PFE': "Pfizer",'ABBV': "Abbvie",'JNJ': "Johnson",
        'SNY': "Sanofi",'MRK': "Merick"}
    
    col = []
    comp_names = []
    col.append('Date')
    data1 = data.copy()
    data_prices1 = data1.iloc[:, 1:].to_numpy()
    
    for i in industry:
        col.extend(ind_dic[i]) 
    data = data[col]
    dates_array = np.array(list(data['Date'].apply(convert_date_to_array)))
    data_prices = data.iloc[:, 1:].to_numpy()# prices for each asset
    dates = np.array(data['Date'])
    
    for i in col:
        try:
            comp_names.append(name_dic[i])
        except:
            pass
    
    # compute expected return and covariance matrix for past 5 years
    day_ind_start0 = 0
    day_ind_end0 = len(dates_array[:,0])
    cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
    mu = np.mean(cur_returns0, axis = 0)
    Q = np.cov(cur_returns0.T)
    day_ind_start = max([i for i, val in enumerate((dates_array[:,0] == 2022) & (dates_array[:,2] == 4)) if val])
    cur_prices = data_prices[day_ind_start,:]
    old_cir_prices = data_prices1[day_ind_start,:]
    
    if len(stocks)==0:
        init_positions = np.zeros(len(data.columns)-1, dtype=int)
        cash_init = invest
    else:
        init_positions = np.zeros(len(data.columns)-1, dtype=int)
        cash_init = np.dot(old_cir_prices, np.array(stocks))
    
    if strategy == 'Maximum Sharpe Ratio Portfolio': 
        x, cash, intrest = strat_max_Sharpe(init_positions, cash_init, mu, Q, cur_prices, trans)
    
    if strategy == 'Equally Weighted Portfolio': 
        x, cash, intrest = strat_equally_weighted(init_positions, cash_init, mu, Q, cur_prices, trans)
    
    if strategy == 'Equal Risk Contributions Portfolio': 
        print(Q)
        x, cash, intrest = strat_equal_risk_contr(init_positions, cash_init, mu, Q, cur_prices, trans)
        
    if strategy == 'Robust Optimization Portfolio': 
        x, cash, intrest = strat_robust_optim(init_positions, cash_init, mu, Q, cur_prices, trans)

    if strategy == 'Mininum Variance Portfolio': 
        x, cash, intrest = strat_min_variance(init_positions, cash_init, mu, Q, cur_prices, trans)

    x_df = pd.DataFrame()
    x_df = x_df.append(pd.DataFrame(x).T)
    x_df.columns = comp_names
    x_df['Cash Remaining ($)'] = round(cash, 2)
    x_df['Total Portfolio Value ($)'] = round(np.dot(cur_prices, x),2)
    x_df.index.names = ['Stocks']
    display(x_df)
    
    pl_df = pd.DataFrame({'Company': comp_names, 'Number of Stocks': x})
    plt.figure(figsize=(10,10))
    plt.pie(pl_df["Number of Stocks"], labels = pl_df["Company"])
    plt.show()
    
    return x_df

## Assistant Initilization

In [133]:
assistant = AssistantV2(version='2021-11-27', authenticator=IAMAuthenticator('7QnyQ0GZgQ6L8dUXt_8m7V2Gh-7GJ8QSYaTEVExESvgI'))
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/b2f00ca9-62c6-4df9-8d18-747e48e6fc6e')
ASSISTANT_ID = "c691812f-b204-4eb2-a190-deb0a9f19301"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

class my_dictionary(dict):

    def __init__(self):
        self = dict()

    def add(self, key, value):
        self[key] = value

ent_dic = my_dictionary()

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
    
                                 input={'message_type': 'text', 'text': text}).get_result()
    entities = []
    for i in range(0,5):
        try:
            entities.append(response['output']['entities'][i]['value'])
        except:
            pass
    try:
        ent_dic.add(response['output']['entities'][0]['entity'], entities)
        entities = []
    except:
        pass
    
    for i in range(0,2):
        try:
            print(response['output']['generic'][i]['text'])
        except:
            pass
  
    try:
        print(response['output']['generic'][0]['title'])
    except:
        pass
    
    if 'options' in response['output']['generic'][0]:
        print("Here are your options, please type them as shown as your response")
        
        for i in range(0,4):
            try:
                print(response['output']['generic'][0]['options'][i]['label'])
            except:
                pass

    return response



## Assistant Interactions

In [141]:
r = message_assistant("hello how are you?")

Hey there! I'm FinBot! I'm here to help you with your portfolio management according to your preferences. 
I can help you either to build a new portfolio or rebalance your own portfolio.
So which service do you want me to do?


In [142]:
r = message_assistant("new portfolio")

Since I am in a development stage right now and you are using a naïve version of me, I can help you build a portfolio which consists of following 25 prominent stocks from 5 different sectors:
Tech: (Google, Apple, Tesla, Amazon, Oracle)
Banking: (CITI, CIBC, RBC, TD, Chase)
Energy: (Enbridge, TC Canada, CNR, Suncor, Imperial Oil)
Automobile: (GM, Honda, Ford, Toyota, Tata)
Pharmaceutical: (Pfizer, AbbVie, Johnson & Johnson, Sanofi, Merck & Co.)
So, let’s begin!! Can you tell me the amount of money you have planned to invest?


In [143]:
r = message_assistant("10000")

And how long are you planning to invest the aforesaid amount?
Here are your options, please type them as shown as your response
6 months
1 year
2 years
3 years


In [144]:
r = message_assistant('3 years')

Ok, can you let me know more about your risk appetite? Just a friendly reminder: Returns on your portfolio are likely proportional to the risk factor associated with it. To make things simple for you, I have three choices for you:
Here are your options, please type them as shown as your response
High risk
Moderate risk
Low risk


In [145]:
r = message_assistant('low risk')

Alright! By the way, do you happen to have any preference for any industries among the following 5: Technology, Banking, Energy, Automobile and Pharmaceutical. 
Do let me know all your preferred industries and I will ensure to avoid stocks from the others…


In [146]:
r = message_assistant('Tech,Auto,Pharmacy,Energy')

For the above-mentioned stocks, how often would you like to rebalance your portfolio?
Here are your options, please type them as shown as your response
Every month
Every 2 months
Quarterly
Half-yearly


In [147]:
r = message_assistant('Every month')

Also, can you let me know what is the transaction cost associated with assets’ trading on your preferred trading platform?  I need this information while rebalancing your portfolio...
(Please enter percentage value)


In [148]:
r = message_assistant('2%')

Are you ok with short selling / leveraging? I will deploy my models based on your choice:
Here are your options, please type them as shown as your response
Yes
No


In [149]:
r = message_assistant('yes')
stocks, invest, industry, strategy, trans = clean_dic(ent_dic)
x_df = protopt(stocks, invest, industry, strategy, trans, data)

Thank you for being so helpful and patient…. Please provide me a few minutes to process your inputs and build a best portfolio for you…


UnboundLocalError: local variable 'time' referenced before assignment